In [1]:
# importing the required libraries
import pandas as pd
import numpy as np
import csv
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
from scipy.stats import norm
import matplotlib.pyplot as plt
from sklearn.svm import SVC
%matplotlib inline
import pickle
import os
import math

Get the absolute path of the folder named "test".<br>
**Warning:** This will still return an absolute path even if the given sub-directory does not exit.

In [2]:
path = os.path.abspath("test")

**Warning:** Might take a long time to run:

In [3]:
# list to store string objects
test_data = []
# store file names
file_name = []
for some in os.listdir(path):
    if some[-4:] == ".txt":
        with open(os.path.join(path,some)) as s:
            data = s.read()
            # tokenize the data
            data = word_tokenize(data)
            # convert to lower case words
            data = [word.lower() for word in data]
            # remove the stop words
            data = [word for word in data if word not in stopwords.words("english")]
            # use porter stemmer and lemmatization
            stemmer = PorterStemmer()
            lemmatizer = WordNetLemmatizer()
            data = [stemmer.stem(word) for word in data]
            data = [lemmatizer.lemmatize(word) for word in data]
            test_data.append(data)
            file_name.append(some)

Loading the vectoriser from memory

In [4]:
vectorier = pickle.load(open("vectoriser", 'rb'))
test_data = [' '.join(words) for words in test_data]

In [5]:
X_test = vectorier.transform(test_data)
# convert sparse to ordinary array
X_test = X_test.toarray()

**Gaussian Naive Bayes:**

In [6]:
mean = pickle.load(open("mean",'rb'))
std = pickle.load(open("std",'rb'))
class_prob = pickle.load(open("class_prob",'rb'))

Prediction:

In [7]:
# store the predicted values
y_pred_nb = []
# two classes
k=2
for x in X_test:
    # here instead of product we are taking sum of log for numerical stability
    y_pred_nb.append(np.argmax([-np.sum(np.log(std[i]) + (x-mean[i])**2/(2*std[i]**2)) + np.log(class_prob[i]) for i in range(k)]))

**Logistic Regression**

In [8]:
beta = pickle.load(open("beta",'rb'))
# add one at the end
X_test_affine = np.hstack([X_test,np.array([1]*len(X_test)).reshape(-1,1)])
y_pred_lr = ((X_test_affine @ beta) > 0)
y_pred_lr = y_pred_lr.astype("int").ravel()

**SVM**

In [9]:
clf = pickle.load(open("svm_clf",'rb'))

In [10]:
y_pred_svm = clf.predict(X_test)

Let us take a vote among the three classifiers:

In [11]:
y_pred_final = (y_pred_lr + y_pred_nb + y_pred_svm)
y_pred_final = (y_pred_final >= 2)
y_pred_final = y_pred_final.astype("int")

In [12]:
file_name

['email1.txt', 'email2.txt']

In [13]:
final_lst_pred = []
for i in range(len(file_name)):
    final_lst_pred.append([file_name[i],y_pred_final[i]])

In [14]:
with open("Predictions.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(final_lst_pred)